# Урок 2. Основные инструменты работы с геоданными. Генерация гео признаков

In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10, 8
%matplotlib inline

ModuleNotFoundError: No module named 'geopandas'

Ноутбук посвящен работе с геоданными с помощью библиотек shapely и geopandas. 

### Знакомство с shapely

`Shapely` - библиотека для работы с геометрическими объектами

`Geopandas` - pandas, только с гео

`folium` - для визуализации

In [ ]:
from shapely.geometry import Point, LineString, Polygon

#### точка

In [ ]:
moscow_lon = 37.618423
moscow_lat = 55.751244
point = Point(moscow_lon, moscow_lat)

In [ ]:
point

У точки как и у любого объекта shaply есть атрибуты площадь и длина:

In [ ]:
print("area ", point.area)
print("length ", point.length)

Граница точки:

In [ ]:
point.bounds

#### Линия

In [ ]:
moscow_lon = 37.618423
moscow_lat = 55.751244

piter_lon = 30.26417
piter_lat = 59.89444

line = LineString([(moscow_lon, moscow_lat), (piter_lon, piter_lat)])

In [ ]:
line

Границы будут отсортированы в следующем порядке: bounds - (minx, miny, maxx, maxy)  - мин долгота, мин широта, макс долгота, макс широта.

In [ ]:
line.bounds

Если линия состоит из набора точек, с помощью атрибута `coords` можно вернуть массив этих точек.

In [ ]:
list(line.coords)

#### Полигон

https://boundingbox.klokantech.com/ - сайт для создания bounding box города

In [ ]:
polygon = Polygon([(37.3193289,55.489927), 
                   (37.9456611,55.489927),
                   (37.9456611,56.009657),
                   (37.3193289,56.009657)])
print(polygon.area)
print(polygon.length)

In [ ]:
polygon

Границы полигона

In [ ]:
polygon.bounds

Для полигона, у которого внутри дырка, можно найти внутренние и внешние границы

In [ ]:
list(polygon.exterior.coords)

In [ ]:
list(polygon.interiors)

### Операции с геометрией

<img src=http://docs.qgis.org/testing/en/_images/overlay_operations.png>

intersection

Создадим 2 точки и буфер вокруг точки, который превращается в круг.

In [ ]:
a = Point(0, 0).buffer(1.1)
b = Point(1, 1).buffer(0.7)

In [ ]:
b

In [ ]:
a.intersection(b)

union

In [ ]:
a.union(b)

In [ ]:
a.difference(b)

contains

Например для того, чтобы проверить лежит ли точка в самом полигоне.

In [ ]:
polygon.contains(Point(moscow_lon, moscow_lat))

In [ ]:
polygon.contains(Point(piter_lon, piter_lat))

### Geopandas

Создаём объект GeoSeries (аналог Series) используя точки из Shapely.

In [ ]:
geo_series = gpd.GeoSeries([Point(-120, 45), Point(-121.2, 46), Point(-122.9, 47.5)])
geo_series

Точки без привязки к местности не имеют смысла, поэтому задаём географическую проекцию.

In [ ]:
geo_series.crs = {'init': 'epsg:4326'}

#### GeoDataFrame

Создадим геодатафрейм на примере датасета административных границ Москвы. Можно подгрузить geojson или esri shape-файл.

In [ ]:
moscow_districts = gpd.read_file('moscow_districts.geojson')

In [ ]:
moscow_districts.columns = moscow_districts.columns.str.lower()

In [ ]:
moscow_districts.head()

Нарисуем геодатафрейм с помощью метода plot и покажем общую численность населения.

In [ ]:
moscow_districts.plot('totalpopul', figsize=(12,10), cmap='OrRd');

#### OpenStreetMap

Скачаем данные по местам из OpenStreetMap на примере Москвы

https://wiki.openstreetmap.org/wiki/Main_Page

Сущности OpenStreeMap:
- `node` - точка, параметры широта и долгота (магазин, офис)
- `way` - линейные объекты, например улица, дорога
- `relation` - отношение, для связи между объектами

In [ ]:
import requests

https://wiki.openstreetmap.org/wiki/Map_Features - информация по категориям

In [ ]:
# источник, откуда будем качать
overpass_url = "https://overpass.kumi.systems/api/interpreter"

# тип выходных данных, которые мы хотим получить. 
#В данном примере это json
# далее указываем сущность - точки с категорией shop и область по которой скачиваем
overpass_query = """
[out:json];
(
 node["shop"](55.4245,37.0919,56.0671,38.1335);
);
out body;
"""

# формируем get-запрос
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [ ]:
data

Преобразуем в датафрейм и оставим только нужные поля

In [ ]:
def get_tag(x, key_name):
    if str(key_name) in x:
        tmp = x[key_name]
        return tmp
    else:
        return "unknown"

In [ ]:
loaded = pd.DataFrame(data['elements'])
loaded['shop'] = loaded['tags'].apply(lambda x: get_tag(x, 'shop'))

In [ ]:
loaded.head()

Но здесь нет поля геометрии. Создадим его.

#### Создание geodataframe

Неважно как называется поле с геометрией, таких полей даже может быть несколько, главное указать какой столбец будет главным в параметре geometry.

In [ ]:
# определяем проекцию для создания наших координат
crs = {'init' :'epsg:4326'}
# создадим лист для хранения массива точек
geometry = [Point(xy) for xy in zip(loaded.lon, loaded.lat)]
# создаём геодатафрейм с указанием исходного датафрейма, проекции и геометрии
places_geo = gpd.GeoDataFrame(loaded, crs=crs, geometry=geometry)

In [ ]:
places_geo.head()

Теперь мы можем связать места с районами Москвы, т.е. посчитать какие места встречаются например в центре. Для этого можно определить атрибутивный join. У нас есть 2 датафрейма и мы их связываем по какому-нибудь общему столбцу. А когда мы имеем дело с геометрией, мы можем воспользоваться spatial join - отношения между объектами.

### Spatial join

проверка проекций - у двух датафреймов они должны совпадать

In [ ]:
moscow_districts.crs

In [ ]:
places_geo.crs

Объединяем 2 датафрейма (сначала районы Москвы, потом сами места), тип inner и отнрошение между геометрическими объектами (intersects, contains, within). 

In [ ]:
places_district = gpd.sjoin(moscow_districts, places_geo, how="inner", op='contains')

На выходе получаем связь районов с местами.

In [ ]:
places_district.head()

Преобразуем датафрейм - сделаем агрегат - сгруппируем места по районам, посчитаем их общее количество.

In [ ]:
places_district['shop_count'] = places_district.groupby('name')['id'].transform('nunique')

В данном датафрейме у нас будет хранится уменьшенная информация.

In [ ]:
district_stats = places_district[['name','geometry', 'shop_count', 'totalpopul']].drop_duplicates('name')#

Нарисуем карту, где у нас будет показано количество магазинов по районам.

In [ ]:
district_stats.plot('shop_count', cmap='OrRd', figsize=(12,10), scheme='equals',legend=True);

Изменим проекцию на Pseudo-Mercator EPSG:3857

In [ ]:
district_stats = district_stats.to_crs(epsg=3857)

In [ ]:
district_stats.plot('shop_count', cmap='OrRd', figsize=(12,10), scheme='equals', legend=True);